In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from pathlib import Path

In [2]:
loader = PyPDFLoader(r"..\testdocs\Calculus.pdf")
data = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [5]:
question = "How to use the division rule?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [6]:
import logging

from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(),
                                                  llm=ChatOpenAI(temperature=0))
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the steps to apply the division rule?', '2. Can you explain the process of utilizing the division rule?', '3. Could you provide a guide on using the division rule effectively?']


7

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())

{'query': 'How to use the division rule?',
 'result': "To use the division rule, you need to follow these steps:\n\n1. Identify the functions involved in the division. Let's say we have a function u(x) divided by a function v(x).\n\n2. Differentiate the numerator function u(x) with respect to x, which means finding its derivative du/dx.\n\n3. Differentiate the denominator function v(x) with respect to x, which means finding its derivative dv/dx.\n\n4. Apply the division rule formula: \n\n   (u(x)/v(x))' = (du/dx * v(x) - u(x) * dv/dx) / (v(x))^2\n\n   This formula represents the derivative of the quotient of two functions.\n\n5. Simplify the expression if possible.\n\nNote: It's important to remember the formula and apply it correctly."}

In [9]:
result = qa_chain({"query": "What is calculus explained elegantly?"})
print(result["result"])

Calculus is a branch of mathematics that deals with change and motion. It involves studying functions and their rates of change, as well as the concept of limits. Calculus allows us to understand and analyze complex phenomena by breaking them down into smaller, more manageable parts. It provides tools and techniques for finding slopes, areas, and rates of change, which are essential in various fields such as physics, engineering, economics, and computer science. In essence, calculus provides a framework for understanding and quantifying change in the world around us.


In [1]:
from langchain.document_loaders import UnstructuredURLLoader, PlaywrightURLLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

url_string = "https://en.wikipedia.org/wiki/Balzan_F.C."

In [5]:
urls

['https://en.wikipedia.org/wiki/Balzan_F.C.']

In [4]:
urls = list(map(str.strip, url_string.split(",")))
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
documents = text_splitter.split_documents(data)

Created a chunk of size 1655, which is longer than the specified 1000


In [6]:
vectorstore = Chroma.from_documents(documents, embeddings)
chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(), vectorstore.as_retriever()
)
chain

NameError: name 'ChatOpenAI' is not defined